In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

Instructions for updating:
Use the retry module or similar alternatives.
hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
exercise = sns.load_dataset('exercise')
exercise.head()

,Unnamed: 0,id,diet,pulse,time,kind
0,0,1,low fat,85,1 min,rest
1,1,1,low fat,85,15 min,rest
2,2,1,low fat,88,30 min,rest
3,3,2,low fat,90,1 min,rest
4,4,2,low fat,92,15 min,rest


In [3]:
exercise['kind'].unique()

[rest, walking, running]
Categories (3, object): [rest, walking, running]

In [4]:
def diet_to_num(record):
    if record == 'low fat':
        return 1
    elif record == 'no fat':
        return 0

In [5]:
def time_to_num(record):
    if record == '1 min':
        return 1
    elif record == '15 min':
        return 15
    elif record == '30 min':
        return 30

In [6]:
exercise['diet_in_num'] = exercise['diet'].apply(diet_to_num)
exercise['time_in_num'] = exercise['time'].apply(time_to_num)
exercise.head()

,Unnamed: 0,id,diet,pulse,time,kind,diet_in_num,time_in_num
0,0,1,low fat,85,1 min,rest,1,1
1,1,1,low fat,85,15 min,rest,1,15
2,2,1,low fat,88,30 min,rest,1,30
3,3,2,low fat,90,1 min,rest,1,1
4,4,2,low fat,92,15 min,rest,1,15


In [7]:
def neural_network_model(input_size, lr):
    network = input_data(shape=[None, input_size], name='state')
    
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    
    
    network = fully_connected(network, 3, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=lr, loss='categorical_crossentropy', name='iris')
    
    model = tflearn.DNN(network, tensorboard_dir='log')
    
    return model
    

In [8]:
def labelize(record):
    if record == 'rest':
        return [1, 0, 0]
    elif record == 'walking':
        return [0, 1, 0]
    elif record == 'running':
        return [0, 0, 1]      

In [9]:
x_train = exercise[['diet_in_num', 'pulse', 'time_in_num']].values
y_train = list(map(labelize, exercise['kind']))

In [10]:
model = neural_network_model(3, 0.00001)
model.fit(x_train, y_train, n_epoch=500, snapshot_step=5, show_metric=True, run_id='flower')

Training Step: 999  | total loss: 0.87790 | time: 0.013s
| Adam | epoch: 500 | loss: 0.87790 - acc: 0.5583 -- iter: 64/90
Training Step: 1000  | total loss: 0.88686 | time: 0.024s
| Adam | epoch: 500 | loss: 0.88686 - acc: 0.5602 -- iter: 90/90
--


In [30]:
predictions = model.predict(x_train[:10])
predictions = list(map(lambda x: x.argmax(), predictions))

In [31]:
def humanize_predictions(predictions):
    humanized_predictions = []
    for prediction in predictions:
        if prediction == 0:
            humanized_predictions.append('rest')
        elif prediction == 1:
            humanized_predictions.append('walking')
        elif prediction == 2:
            humanized_predictions.append('running')
    return humanized_predictions

In [32]:
humanize_predictions(predictions)

['walking',
 'rest',
 'rest',
 'walking',
 'walking',
 'rest',
 'walking',
 'walking',
 'rest',
 'rest']